In [1]:
# "Import modules"

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from bs4 import BeautifulSoup
from lxml import etree
import random
#import lei
from selenium.webdriver.common.action_chains import ActionChains

def DateSelection(start_date, end_date, driver):
    datepicker1 = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@id='fromDatepicker']")))
    datepicker1.send_keys(Keys.CONTROL + "a")
    datepicker1.send_keys(start_date)
    datepicker2 = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='toDatepicker']")))
    datepicker2.send_keys(Keys.CONTROL + "a")
    datepicker2.send_keys(end_date)
    datepicker2.send_keys(Keys.ENTER)
    print('Dates Selected Successfully')
    time.sleep(20)


def Expand_Table(driver, Id):
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, Id))).click()
    time.sleep(18)
    WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="list"]/tbody/tr[position()>1]')))
    print('Table Expanded')


def Scrap_table(number_of_table_pages):
    i = 576

    table_template = pd.DataFrame(
        columns=['Company', 'Ticker','Country', 'SecurityID', 'MeetingDate', 'Type'])

    table_details_template = pd.DataFrame(
         columns=['Item#', 'Proposal', 'Proponent', 'Mgmt Rec', 'Vote', 'Company', 'Ticker', 'MeetingDate', 'recordDate',
                  'meetingType', 'industrySector', 'country'])


    if input('Enter Y if you want to scrap the meeting tables (Be aware of T&Cs): ').lower() == 'y':
        details_answer = input('Enter Y if you want to scrap the meeting details:').lower()

        while i <= int(number_of_table_pages):
            print(f'Changing table page to: {i} ')
            dropdown = Select(driver.find_element(By.XPATH, '//*[@id="PageDropdown"]'))
            dropdown.select_by_value(str(i))
            i += 1
            time.sleep(random.randint(15, 22))
            driver.execute_script("window.scrollBy(0,-300)", "")
            print(f'Waiting')
            WebDriverWait(driver, 20).until(
                EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="list"]/tbody/tr[position()>1]')))
            table = driver.find_elements(By.XPATH, '//*[@id="list"]/tbody/tr[position()>1]')
            lista = []
            tabla_iterator = 0

            for element in table:
                dicto = dict()
                dicto['Company'] = element.find_element(By.XPATH, './td[@aria-describedby="list_CompanyName"]').text
                print(dicto['Company'])
                dicto['Ticker'] = element.find_element(By.XPATH, './td[@aria-describedby="list_Ticker"]').text
                dicto['Country'] = element.find_element(By.XPATH, './td[@aria-describedby="list_Country"]').text
                dicto['SecurityID'] = element.find_element(By.XPATH, './td[@aria-describedby="list_SecurityID"]').text
                dicto['MeetingDate'] = element.find_element(By.XPATH, './td[@aria-describedby="list_MeetingDate"]').text
                dicto['Type'] = element.find_element(By.XPATH, './td[@aria-describedby="list_MeetingType"]').text
                #dicto['Status'] = element.find_element(By.XPATH, './td[@aria-describedby="list_VoteFlag"]').text
                #dicto['FundName'] = element.find_element(By.XPATH, './td[@aria-describedby="list_FundName"]').text
                lista_detalles = []
                lista.append(dicto)
                if tabla_iterator > 6:
                    driver.execute_script("window.scrollBy(0,100)", "")

                if details_answer == 'y':
                    source = driver.find_elements(By.XPATH, '//*[@id="list"]/tbody/tr[position()>1]')[tabla_iterator]
                    action = ActionChains(driver)
                    action.move_to_element(source).click().perform()
                    #driver.find_elements(By.XPATH, '//*[@id="list"]/tbody/tr[position()>1]')[tabla_iterator].click()
                    driver.execute_script("window.scrollBy(0,50)", "")
                    print("Scraping Details")
                    dicto2 = dict()
                    time.sleep(random.randint(10, 13))
                    dicto2['Ticker'] = driver.find_element(By.ID, 'ticker').text
                    dicto2['MeetingDate'] = driver.find_element(By.ID, 'meetingDate').text
                    dicto2['recordDate'] = driver.find_element(By.ID, 'recordDate').text
                    dicto2['meetingType'] = driver.find_element(By.ID, 'meetingType').text
                    dicto2['industrySector'] = driver.find_element(By.ID, 'industrySector').text
                    dicto2['country'] = driver.find_element(By.ID, 'country').text
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    dom = etree.HTML(str(soup))
                    table2 = dom.xpath('//*[@id="listDetail"]/tbody/tr[position()>1]')
                    tabla_iterator += 1

                    for element2 in table2:
                        dicto3 = dict()
                        dicto3['Item#'] = element2[1].text
                        dicto3['Proposal'] = element2[2].text
                        dicto3['Proponent'] = element2[3].text
                        dicto3['Mgmt Rec'] = element2[4].text
                        dicto3['Vote'] = element2[5].text
                        dicto3['Company'] = dicto['Company']
                        dicto3['Ticker'] = dicto2['Ticker']
                        dicto3['MeetingDate'] = dicto2['MeetingDate']
                        dicto3['recordDate'] = dicto2['recordDate']
                        dicto3['meetingType'] = dicto2['meetingType']
                        dicto3['industrySector'] = dicto2['industrySector']
                        dicto3['country'] = dicto2['country']
                        lista_detalles.append(dicto3)
                        driver.execute_script("window.scrollBy(0,6)", "")

                    #driver.execute_script("arguments[0].scrollIntoView();",
                    #                      driver.find_element(By.XPATH, '// *[ @ id = "expandCollapseType_list"]'))
                    driver.execute_script("window.scrollBy(0,-84)", "")
                    #driver.find_element(By.XPATH,
                    #                    '//*[@id="meetingListGrid"]/div[1]/table/tbody/tr/td[1]/button').click()
                    #
                    driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH,
                                        '//*[@id="meetingListGrid"]/div[1]/table/tbody/tr/td[1]/button'))

                    time.sleep(random.randint(4, 9))

                    table_details_template = pd.concat([table_details_template, pd.DataFrame(lista_detalles)],
                                                       ignore_index=True)
                    table_details_template.to_excel('TableDetailsUBS_2023_576.xlsx')

            table_template = pd.concat([table_template, pd.DataFrame(lista)], ignore_index=True)

            print('Saving to Excel')
            table_template.to_excel('TableReadyUBS_2023_576.xlsx')

    else:
        input('Press Enter to exit.')


if __name__ == "__main__":
    "Dates to scrap"
    start_date = '01-Jan-2023'
    end_date = '30-Sep-2023'

    "Intialize driver and load webpage"
    "************************PLEASE CHANGE DIRECTORY TO YOUR DRIVER PATH*******************************"
    #driver_path = "/usr/local/bin/chromedriver"
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get('https://vds.issgovernance.com/vds/#/MjU0/')
    element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//input[@id='fromDatepicker']")))

    "Pass dates to function"
    DateSelection(start_date, end_date, driver)


    "Expand table and get number of pages in table"

    #Expand_Table(driver, "expandCollapseType_list")
    number_of_table_pages = driver.find_element(By.XPATH, '//*[@id="PageDropdown"]/..').text.split()[-1]

    "Calling Scrap Function"
    Scrap_table(number_of_table_pages)


Dates Selected Successfully


In [56]:
import requests
import json
import pandas as pd
def getdata(start_date, end_date, rows, page):
    cookies = {
        'BIGipServervds2-app.prod.us.issapps.com_8330': '2483286538.35360.0000',
        'BIGipServervds.issgovernance.com_8120': '889319946.47135.0000',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-GB,en;q=0.5',
        # 'Accept-Encoding': 'gzip, deflate, br',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection': 'keep-alive',
        'Referer': 'https://vds.issgovernance.com/vds/',
        # 'Cookie': 'BIGipServervds2-app.prod.us.issapps.com_8330=2483286538.35360.0000; BIGipServervds.issgovernance.com_8120=889319946.47135.0000',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
    }

    params = {
        'customerID': 'MjU0',
        # 'fromDate': '2023-01-01',
        # 'toDate': '2023-09-30',
        'fromDate': start_date,
        'toDate': end_date,
        'liveSiteYN': '1',
        'random': '0.49761613755807876',
        'locale': 'en',
        'MeetingTypeList': '',
        'CountryList': '',
        'VotedList': '',
        '_search': 'false',
        'nd': '1705577300279',
        'rows': rows,
        'page': page,
        'SortByColumn': 'CompanyName',
        'OrderBy': 'asc',
    }

    response = requests.get('https://vds.issgovernance.com/vds/api/getVdsData/14', params=params, headers=headers)
    return response.json()['data']



def getCompanydata(MultipleFundIDs, MeetingId):
    cookies = {
        'BIGipServervds2-app.prod.us.issapps.com_8330': '2483286538.35360.0000',
        'BIGipServervds.issgovernance.com_8120': '889319946.47135.0000',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-GB,en;q=0.5',
        # 'Accept-Encoding': 'gzip, deflate, br',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection': 'keep-alive',
        'Referer': 'https://vds.issgovernance.com/vds/',
        # 'Cookie': 'BIGipServervds2-app.prod.us.issapps.com_8330=2483286538.35360.0000; BIGipServervds.issgovernance.com_8120=889319946.47135.0000',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
    }

    params = {
        'customerID': 'MjU0',
        'fromDate': '',
        'toDate': '',
        'fundValue': MultipleFundIDs,
        'liveSiteYN': '1',
        'meetingID': MeetingId,
        'random': '0.956757217578498',
        'locale': 'en',
        '_search': 'false',
        'nd': '1705577418341',
        'rows': '2000',
        'page': '1',
        'SortByColumn': 'Proposal',
        'OrderBy': 'asc',
    }

    response = requests.get('https://vds.issgovernance.com/vds/api/getVdsData/7', params=params, cookies=cookies, headers=headers)
    response.json()['data']
def getTotalrow(start_date, end_date, rows, page):
     TRow= getdata(start_date, end_date, rows, page)[0]['TotalRows']
     return TRow
    

In [49]:
start_date = '2023-01-01'
end_date = '2023-09-30'
rows = getTotalrow(start_date, end_date, 20, 1)
data = getdata(start_date, end_date, rows, 1)


In [55]:
ResultData = []
for i in data:
    dataDict = {"CompanyName": i['CompanyName'], "Ticker": i['Ticker'], "Country": i['Country'], "SecurityID": i['SecurityID'], "MeetingDate": i['MeetingDate'], "MeetingType": i['MeetingType'], "FundName": i['FundName'], "MultipleFundIDs": i['MultipleFundIDs'], "MeetingID": i['MeetingID'] }
    print(dataDict)
    companyData = getCompanydata(dataDict['MultipleFundIDs'], dataDict['MeetingID'])
    print(companyData)

{'CompanyName': '1&1 AG', 'Ticker': '1U1', 'Country': 'Germany', 'SecurityID': 'D5934B101', 'MeetingDate': '2023-05-16 00:00:00.0', 'MeetingType': 'Annual', 'FundName': '', 'MultipleFundIDs': '39945', 'MeetingID': -4766008}
[{'FundFootnoteSymbol': None, 'MeetingTypeDetail': 'Annual', 'CompanyID': 0, 'ShareholderProposal': 0, 'FundFootnoteText': None, 'FundNames': 'UBS', 'SeqNumber': 1, 'MeetingFootnoteSymbol': '', 'SecurityIDDetail': 'D5934B101', 'MeetingID': -4766008, 'EsgPillar': '', 'MgtRecVote': None, 'RecordDateDetail': '2023-04-24 00:00:00.0', 'Notes': 'NA', 'TickerDetail': '1U1', 'ProposalSubCategory': '', 'ProposalFootnoteText': '', 'CountryDetail': 'Germany', 'SignificantProposalYN': '', 'ClientVoteList': 'Non Voting', 'MeetingFootnoteText': '', 'ProposalCategory': '', 'SharesVotedList': '0', 'BallotItemNumber': '1', 'VoteResult': None, 'MeetingDateDetail': '2023-05-16 00:00:00.0', 'ItemOnAgendaID': 1, 'CompanyNameDetail': '1&1 AG', 'SixDigitSectorType': None, 'ProposalFootnot

KeyboardInterrupt: 